In [1]:
from yolo import yolo
from save import SaveNumpy
import cv2
from pathlib import Path
from submodules import cv3

In [ ]:
video_path = Path(r"C:\Users\articulusaitraining\projects\ptg\punthai\videos\29\D01_20250529063837_re-encode.mp4")
model_path = Path(r"C:\Users\articulusaitraining\projects\ptg\punthai\weights\v1x_PunthaiChaengWatthana_freeze10.pt")
save_as = r"."

Save Predicted npy

In [14]:
model = yolo(model_path)
saver = SaveNumpy(save_as=save_as)

cap=cv2.VideoCapture(video_path)
saved_path = None

config = {
    'path':{
        'model_path':model_path.as_posix(),
        'source_path':video_path.as_posix(),
    },
    'model':model_path.name,
    'source':video_path.name,
    'cam_prop':{
        'fps':round(cap.get(cv2.CAP_PROP_FPS)),
        'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    },
    'predictor':{
        'names':model.names
    }
}

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        result = model.predict_and_save(frame, verbose=False)[0]
        boxes = result.boxes.xyxyn.cpu().numpy()
        saved_path = saver.npy(arr=boxes, cfg=config)
        if cv3.imshow(result.plot()).interrupt(1):
            break
finally:
    cap.release()
    cv3.destroyAllWindows()

    if saved_path:
        config['predictor']['predictor_params'] = model._predictor_params
        saver.save_config(config, save_as=saved_path.parent.parent)

Save Predicted npz

In [ ]:
model = yolo(model_path)
saver = SaveNumpy(save_as=save_as)

cap=cv2.VideoCapture(video_path)
saved_path = None

config = {
    'path':{
        'model_path':model_path.as_posix(),
        'source_path':video_path.as_posix(),
    },
    'model':model_path.name,
    'source':video_path.name,
    'cam_prop':{
        'fps':round(cap.get(cv2.CAP_PROP_FPS)),
        'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    },
    'predictor':{
        'names':model.names
    }
}

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        result = model.predict_and_save(frame, verbose=False)[0]
        boxes = result.boxes.xyxyn.cpu().numpy()
        conf = result.boxes.conf.cpu().numpy()
        cls = result.boxes.cls.cpu().numpy()
        saved_path = saver.npz(cfg=config, boxes=boxes, conf=conf, cls=cls)
        if cv3.imshow(result.plot()).interrupt(1):
            break
finally:
    cap.release()
    cv3.destroyAllWindows()

    if saved_path:
        config['predictor']['predictor_params'] = model._predictor_params
        saver.save_config(config, save_as=saved_path.parent.parent)